In [14]:
'''
    Load RESNET model and split it
        1. layer by layer
        2. [TODO] vertically 
'''

'\n    Load RESNET model and split it\n        1. layer by layer\n        2. [TODO] vertically \n'

In [15]:
from source.core.engine import MoP
from source.core import run_partition
from os import environ
from source.utils.dataset import *
from source.utils.misc import *
from source.utils.split_network import *

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from source.models import resnet
from source.core.split_manager import SplitManager

import torch.nn.functional as F

import numpy as np

from source.utils import io
from source.utils import testers
from source.core import engine

import json
import itertools

from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from torchsummary import summary

import time

In [16]:
# setup config
dataset='cifar10'
environ["config"] = f"config/{dataset}.yaml"

configs = run_p.main()

configs["device"] = "cpu"
configs['load_model'] = "cifar10-resnet18-kernel-npv0-pr0.75-lcm0.001.pt"
configs["num_partition"] = '4' #'resnet18-v2.yaml'

device :  
model :  resnet18
data_code :  cifar10
num_classes :  10
model_file :  test.pt
epochs :  0
batch_size :  128
optimizer :  sgd
lr_scheduler :  default
learning_rate :  0.01
seed :  1234
sparsity_type :  kernel
prune_ratio :  1
admm :  True
admm_epochs :  300
rho :  0.0001
multi_rho :  True
retrain_bs :  128
retrain_lr :  0.005
retrain_ep :  50
retrain_opt :  default
xentropy_weight :  1.0
warmup :  False
warmup_lr :  0.001
warmup_epochs :  10
mix_up :  True
alpha :  0.3
smooth :  False
smooth_eps :  0
save_last_model_only :  False
num_partition :  1
layer_type :  regular
bn_type :  masked
par_first_layer :  False
comm_outsize :  False
lambda_comm :  0
lambda_comp :  0
distill_model :  
distill_loss :  kl
distill_temp :  30
distill_alpha :  1


In [17]:
# load data and load or train model
model = get_model_from_code(configs).to(configs['device']) # grabs model architecture from ./source/models/escnet.py
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [18]:
# load weights into full model
state_dict = torch.load(io.get_model_path("{}".format(configs["load_model"])), map_location=configs['device'])
model = io.load_state_dict(model, 
                    state_dict['model_state_dict'] if 'model_state_dict' in state_dict 
                    else state_dict['state_dict'] if 'state_dict' in state_dict else state_dict,)


In [19]:
'''
    add partitions and communications to configs
'''

# gets random test input (with correct size)
input_var = engine.get_input_from_code(configs)
#print(input_var)

# Config partitions and prune_ratio
configs = engine.partition_generator(configs, model)
            
# Compute output size of each layer
configs['partition'] = engine.featuremap_summary(model, configs['partition'], input_var)

# split model general parameters

# make copies of model per machine
num_machines = max(configs['partition']['bn_partition']) # TODO: double check this makes sense
model_machines = [model]*num_machines

layer_names_fx =  get_graph_node_names(model)[1]
total_layers_fx = len(layer_names_fx)

split_module_names = list(configs['partition'].keys())

print(layer_names_fx)


Inference time per data is 32.542229ms.
conv1.weight 1024
layer1.0.conv1.weight 1024
layer1.0.conv2.weight 1024
layer1.1.conv1.weight 1024
layer1.1.conv2.weight 1024
layer2.0.conv1.weight 256
layer2.0.conv2.weight 256
layer2.0.shortcut.0.weight 256
layer2.1.conv1.weight 256
layer2.1.conv2.weight 256
layer3.0.conv1.weight 64
layer3.0.conv2.weight 64
layer3.0.shortcut.0.weight 64
layer3.1.conv1.weight 64
layer3.1.conv2.weight 64
layer4.0.conv1.weight 16
layer4.0.conv2.weight 16
layer4.0.shortcut.0.weight 16
layer4.1.conv1.weight 16
layer4.1.conv2.weight 16
['x', 'conv1', 'bn1', 'relu', 'layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.relu', 'layer1.0.conv2', 'layer1.0.bn2', 'layer1.0.add', 'layer1.0.relu_1', 'layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.relu', 'layer1.1.conv2', 'layer1.1.bn2', 'layer1.1.add', 'layer1.1.relu_1', 'layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.relu', 'layer2.0.conv2', 'layer2.0.bn2', 'layer2.0.shortcut.0', 'layer2.0.shortcut.1', 'layer2.0.add', 'layer2.0.relu_1', 'la

In [20]:
''' 
    IO helpers
'''

def prep_data(tensor, node, layer):
    '''
        Prepare input tensors to model 
    '''
    return {
        'node' : node,
        'layer' : layer,
        'tensor' : tensor, 
        'Cin' : list(get_nonzero_channels(tensor)),
        'is_empty' : False
    }

def compare_outputs_wrapper(horz_output, comm_layer, split_model_output, limit = 0.1):
    '''
        Checks if output from split model is correct 

        return 0 if check fails and 1 if passes
    '''
    
    if comm_layer == 0:
        print(f'Input layer. Skipping comparison')
        return 1
    else:
        # if reaches the end use the full model output 
        if comm_layer == len(horz_output):
            comm_layer_name = 'x'
        else:
            comm_layer_name = layer_names_fx[comm_layer]
    
        truth_output = horz_output[comm_layer_name]
        if torch.is_tensor(truth_output):
            
            max_diff, max_by_Cout = compare_outputs(split_model_output, truth_output)
            if max_diff > 0.1:
                return 0
            else:
                return 1 
        else:
            print(f'Horizontal output is {type(truth_output)}. Skipping comparison')
            return 1


In [21]:
''' Prep Inout for Mock Run version 2 (fx)'''

# TODO: reduce size of communicated tensors to only what is necessary 
# TODO: also check bias for nonzero
# TODO: implement broadcast vs multi-cast? switch e.g. a single machine broadcasts outputs because weights that reach across layers are executed on the rx machine only 
# TODO: handle arbitrary kernel to machine assignment 

# channel_id == INPUTS
# filter_id  == OUTPUTS

configs_copy = configs # keep the origonal config 

# setup input 
N_batch = 1
torch.manual_seed(0)
input_tensor = torch.rand(N_batch, 3, 32, 32, dtype=torch.float64, device=torch.device(configs_copy['device'])) # 1k images, 3 channels, 32x32 image (cifar100) 
model = model.type(torch.float64)

# add logic for final layer TODO: add this in automatically somewhere
final_node= 0
linear_map = SplitManager.get_io_for_linear(configs_copy, layer_names_fx, num_machines, 0, 10)
configs_copy['partition']['linear.weight'] = linear_map
configs_copy['partition']['FINAL_MODEL_OUTPUT.weight'] = { 'channel_id' : [np.array([])]*num_machines }
configs_copy['partition']['FINAL_MODEL_OUTPUT.weight']['channel_id'][0] = np.arange(model.linear.out_features) # send all outputs to machine 0 

# make SplitManagers for split model execution 
debug = True
configs_copy['dtype'] = 'float64'
split_managers = [SplitManager]*num_machines
for i in range(num_machines):
    split_managers[i] = SplitManager(configs_copy, i, num_machines, input_tensor, debug=True)

# emulates queue that server collects inputs with. 
# Each network node has it's own list of inputs it looks at when collecting inputs 
# input :
#   [rx node (str)] : (list) list of data dicts 
input = {}
for imach in range(num_machines):
    in_data = prep_data(input_tensor, -1, 0)
    input[str(imach)] = [in_data]

# put models into eval mode and on device
model.eval()
model.to(configs['device'])

# get true output at each layer
# WARNING: bn layers are initialized differently between model in this notebook and model in split manager. The moving mean and variance fields do not match HOWEVER the weights and biases are the same
#horz_output, layer_output_size_LUT = get_output_at_each_layer(model, input_tensor) 
horz_output, layer_output_size_LUT = get_output_at_each_layer(split_managers[0].model, input_tensor)

'''
    mock run through inference using split models 
'''

# timing
split_execution_start_time = time.time()
layer_completion_time_stamp = {}
layer_execution_duration = {}

# make inference 
with torch.no_grad():

    # iterate through layers 1 module at a time 
    while True:#range(num_total_modules): # 16 <=> layer_1 block 

        # Write outputs to one big queue with everyone's data in it 
        output = {}

        # iterate through each machine (done in parallel later)
        for imach in range(num_machines):
            
            if split_managers[imach].is_done():
                print(f'\tMachine {split_managers[imach].machine} has finished calculations. Skipping...\n')
                continue
            else:
                print(f'\tExecuting on machine {imach}')
            
            # collect communication inputs if necessary 
            # 1. collect inputs for this machine and it's current layer
            # 2. add them to the current_tensor 
            if str(imach) in input:
                success = split_managers[imach].process_input(input[str(imach)]) # update local tensor with inputs
                if success == -1:
                    print(f'\t\tNo comms for {imach} layer {split_managers[imach].current_layer} yet. Skipping...\n')
                    continue
            
            # execute split layers
            output_tensor = split_managers[imach].execute_layers_until_comms()

            # prep output
            processed_output = split_managers[imach].prep_output(output_tensor) # prepare communication
            
            # Send output! (Append output data to proper rx machine queue)
            for a_output in processed_output:
                rx_mach = a_output['node_to']
                if str(rx_mach) in output:
                    output[str(rx_mach)].append(a_output)
                else:
                    output[str(rx_mach)] = [a_output]

        # are machines finished executing model?
        if all([amanager.is_done() for amanager in split_managers]):
            # final execution time
            tot_split_execution_time = time.time() - split_execution_start_time
            print(f'\n\n############################# FINAL EXECUTION TIME {tot_split_execution_time} [seconds] #############################\n\n')
            break

        # check output after each communication/convolutional layer 
        all_output = [el for alist in list(output.values()) for el in alist]
        comm_layers = np.array([adict['layer'] for adict in all_output])
        comm_layer = np.unique(comm_layers)
        if len(comm_layer) > 1:
            print('ERROR: MORE THAN ONE COMM LAYER IN OUTPUT')
            break
        else:
            comm_layer = int(comm_layer)
        comm_layer_name = layer_names_fx[comm_layer]
        
        print(f'\n\n---------------------------FINISHED COMMS FOR {comm_layer_name}-----------------------------------\n\n')

        if not comm_layer_name == 'linear':
            # collect communicated output
            split_model_output = SplitManager.combine_all_dict_inputs(
                split_managers[0].get_input_size(comm_layer), 
                all_output, # TODO: assumes you can sum all outputs from the same layer 
                comm_layer, 
                device=split_managers[0].device,
                dtype=split_managers[0].dtype)

            # add current tensors stored in machines 
            for i in range(num_machines):
                if split_managers[i].current_layer-1 == comm_layer:
                    split_model_output = split_model_output + split_managers[i].current_tensor

            # compare split vs full (truth) 
            print(f'\nChecking split model output for {comm_layer_name}:')
            compare_outputs_wrapper(horz_output, comm_layer, split_model_output)
            print('\n\n')

        # send to next layer  
        input = output


	Executing on machine 0
		No comms for 0 layer 2 yet. Skipping...

	Executing on machine 1
		 current input channels tensor([0, 1, 2])
		 EXEC SPLIT: #1-conv1; Shape=[1, 64, 32, 32]; C_out=[16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]
		 INITIALIZING CURRENT TENSOR FOR: #2-bn1; Shape=[1, 64, 32, 32]; C_in=[16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]
	Executing on machine 2
		 current input channels tensor([0, 1, 2])
		 EXEC SPLIT: #1-conv1; Shape=[1, 64, 32, 32]; C_out=[32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
		 INITIALIZING CURRENT TENSOR FOR: #2-bn1; Shape=[1, 64, 32, 32]; C_in=[32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
	Executing on machine 3
		 current input channels tensor([0, 1, 2])
		 EXEC SPLIT: #1-conv1; Shape=[1, 64, 32, 32]; C_out=[48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]
		 INITIALIZING CURRENT TENSOR FOR: #2-bn1; Shape=[1, 64, 32, 32]; C_in=[48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]


---------------------------FINISHED COMMS FOR con

C:\Users\natet\AppData\Local\Temp\ipykernel_31072\1610696366.py:116: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  comm_layer = int(comm_layer)


		 EXEC SPLIT: #2-bn1; Shape=[1, 64, 32, 32]; C_out=[32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
		 INITIALIZING CURRENT TENSOR FOR: #3-relu; Shape=[1, 64, 32, 32]; C_in=[32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
		Checking output from bn1 C_in [32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]: 




		 current input channels tensor([32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47])
		-Applying ReLU
		 INITIALIZING CURRENT TENSOR FOR: #4-layer1.0.conv1; Shape=[1, 64, 32, 32]; C_in=[32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
		Checking output from relu C_in [32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]: 




		 current input channels tensor([32, 35, 36, 37, 38, 39, 42, 43, 44, 46, 47])
		-Saving input for later...
		 EXEC SPLIT: #4-layer1.0.conv1; Shape=[1, 64, 32, 32]; C_out=[ 0  3  6 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 53]
		 INITIALIZING CURRENT TENSOR FOR: #5-layer1.0.bn1; Shape=[1, 64, 32, 32]; C_in=[32 33 34 35 36 37 38 39 40 41 42 4